In [2]:
# Proyecto Final Análisis Aplicado - Aimeé León - 142559

import numpy as np
print(np.__version__)

1.19.2


In [3]:
# Función de Prueba de los Algoritmos (Rosenbrock)
def Rosenbrock(x):
    # Aquí suponemos que x ya es una matriz (vector) de nx1
    return (1-x[0,0])**2 + 100*(x[1,0]-x[0,0]**2)**2

In [4]:
def gradiente(func,xk):
    # Código que calcula el gradiente de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk array y lo convertimos a matriz de nx1
    cambio = .000001
    n = len(xk)
    grad = np.matrix(np.zeros((n,1)))
    
    for i in range(1,n+1):
        aux = xk.copy()
        aux[i-1,0] = aux[i-1,0] + cambio
        grad[i-1,0] = round((func(aux) - func(xk))/cambio,2)
    
    return grad

In [5]:
def hessiana(func,xk):
    # Código que calcula la hessiana de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk matrix de nx1
    cambio = .000001
    n = len(xk)
    hess = np.matrix(np.zeros((n,n)))
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i==j:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux2[i-1,0] = aux2[i-1,0] - cambio
                hess[i-1,j-1] = round((func(aux1)-2*func(xk)+func(aux2))/cambio**2,2)
            else:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux3 = xk.copy()
                
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux1[j-1,0] = aux1[j-1,0] + cambio
                
                aux2[j-1,0] = aux2[j-1,0] + cambio
                
                aux3[i-1,0] = aux3[i-1,0] + cambio
                
                hess[i-1,j-1] = round((func(aux1)-func(aux2)-func(aux3)+func(xk))/cambio**2,2)
                
    return hess

In [6]:
# Algoritmo 3.1: Backtracking Line Search

def BLS(func,xk):
    a = 1
    # Pues es la alpha que corresponde a la dirección de Newton
    c = 10**(-4)
    pk = -np.linalg.inv(hessiana(func,xk))*gradiente(func,xk)
    while func(xk+a*pk) > (func(xk) + c*a*np.transpose(gradiente(func,xk))*pk):
        a = .8*a
    return a

In [7]:
def posdef(func,xk):
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    if all(eigen > 0):
        r = 1 # cumple hessiana positiva definida
    else:
        r = 0 # no cumple hessiana positiva definida
        
    return r

In [8]:
def menor_eigen(func,xk):
    # Método que devuelve el menor de los eigenvalores de una matriz multiplicado por -1
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    n = len(xk)
    min = 0
    for i in range(1,n+1):
        if eigen[i-1] < min:
            min = eigen[i-1]
    min = -min
    return min

In [9]:
# Método que devuelve el factor que le sumaremos a la Hessiana para volverla positiva definida

def factorCholesky(A,n):  
    b = .0001
    if np.min(np.diag(A))>0:
        fact = 0
    else:
        fact = -np.min(np.diag(A)) + b
    
    for i in range(1,100):
        eigen = np.linalg.eig(A + fact*np.identity(n))[0]
        if all(eigen > 0):
            r = 1
        else:
            r = 0

        if r==0: # es decir, si aún no es positiva definida
            fact = np.max([b,2*fact])
    
    return fact

In [10]:
# Algoritmo 3.2: Line Search Newton with Hessian Modification

def LSNM(func,x0):
    xk =  x0
    n = len(xk)
    Bk = hessiana(func,xk)
    
    for k in range(1,100):
        s = factorCholesky(Bk,n)
        Bk = hessiana(func,xk) + s*np.identity(n)
        pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
        xk = xk + BLS(func,xk)*pk    
            
    return xk

In [11]:
# Broyden, Fletcher, Goldfarb, Shanno
def BFGS(func,xk):
    n = len(xk)
    e = .00001
    Hk = np.linalg.inv(hessiana(func,xk))
    
    while np.linalg.norm(gradiente(func,xk))>e:
        pk = -Hk*gradiente(func,xk)  #dirección
        xk1 = xk + BLS(func,xk)*pk
        sk = xk1 - xk
        yk = gradiente(func,xk1) - gradiente(func,xk)
        rok = float(1/(np.transpose(yk)*sk))
        Hk1 = (np.identity(n)-rok*sk*np.transpose(yk)) * Hk * (np.identity(n)-rok*yk*np.transpose(sk)) + rok*sk*np.transpose(sk)
        xk = xk1
        Hk = Hk1
    
    return xk

In [23]:
def AN(func,x0): # Algoritmo de Newton
    xk =  x0
    n = len(xk)
    a = 1
    
    for k in range(1,100):
        Bk = hessiana(func,xk)
        pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
        xk = xk + a*pk    
            
    return xk

In [24]:
def LS(func,x0): # Búsqueda Lineal Base (Sin modificaciones a la Hessiana)
    xk =  x0
    n = len(xk)
    
    for k in range(1,100):
        Bk = hessiana(func,xk)
        pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
        xk = xk + BLS(func,xk)*pk    
            
    return xk

In [25]:
xk = np.matrix([[1.5],[2.3]])

#LSNM(Rosenbrock,xk)
#BFGS(Rosenbrock,xk)
#LS(Rosenbrock,xk)
LS(Rosenbrock,xk)

matrix([[1.00304748],
        [1.00611739]])